In [8]:
import torch
import pykeen
import pandas as pd
from pykeen import predict
from pykeen.pipeline import pipeline
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
import numpy as np
from pykeen.utils import set_random_seed


file_path = 'C.1_query.tsv'
tf = TriplesFactory.from_path(file_path, delimiter="\t")
training, testing = tf.split([0.85, 0.15], random_state=2025)
training, validation = training.split([0.8, 0.2], random_state=2025)
device = "cuda" if torch.cuda.is_available() else "cpu"

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [12102, 2463]
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [9311, 2791]


# Model Hyperparameter Tuning

In [ ]:
def run_pipeline(model_name, model_kwargs_range, study_name, random_seed=2025):
    # Set the random seed for reproducibility
    set_random_seed(random_seed)

    # Validate input
    if model_name is None or model_kwargs_range is None:
        raise ValueError("Both model name and model parameter ranges must be provided.")

    # Run the hyperparameter optimization pipeline
    result = hpo_pipeline(
        training=training,
        testing=testing,
        validation=validation,

        # Hyperparameter optimization configuration
        sampler="tpe",
        n_trials=30,

        study_name=study_name,

        # Training parameter ranges
        training_kwargs_ranges=dict(
            num_epochs=dict(type="int", low=50, high=150, step=50)
        ),

        # Model and its parameter search space
        model=model_name,
        model_kwargs=dict(
            random_seed=random_seed
        ),
        model_kwargs_ranges=model_kwargs_range,

        # Optimizer configuration
        optimizer="adam",
        optimizer_kwargs_ranges=dict(
            lr=dict(type="float", low=0.0001, high=0.001, log=True),
            weight_decay=dict(type="float", low=1e-4, high=1e-3, log=True)
        ),

        # Negative sampling configuration
        negative_sampler="basic",
        negative_sampler_kwargs_ranges=dict(
            num_negs_per_pos=dict(type="int", low=1, high=10, step=3)
        ),

        # Early stopping settings
        stopper="early",
        stopper_kwargs=dict(
            patience=10,
            frequency=5,
            metric="hits@5",
            relative_delta=0.002,
            larger_is_better=True,
        ),

        device=device,
    )

    return result

# TransH Model

In [3]:
model_name = "TransH"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64)
        )

# Run the HPO pipeline
resultTransH = run_pipeline(model_name, model_kwargs_range, 'TransH_HPO_Experiment')

[I 2025-05-27 23:41:44,527] A new study created in memory with name: TransH_HPO_Experiment
No random seed is specified. Setting to 2019358053.


Training epochs on cuda:0:   0%|          | 0/150 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 3.38s seconds
INFO:pykeen.hpo.hpo:Pruned trial: <optuna.trial._trial.Trial object at 0x0000019105CDB690> at epoch 5 due to both.realistic.inverse_harmonic_mean_rank=0.044428520243640274
[I 2025-05-28 00:32:11,567] Trial 29 pruned. 


In [4]:
resultTransH.save_to_directory('transh_hpo_result')

# RotatE Model

In [5]:
model_name = "RotatE"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64),
        )
# Run the HPO pipeline
resultRotatE = run_pipeline(model_name, model_kwargs_range, 'RotatE_HPO_Experiment')

[I 2025-05-28 00:32:11,715] A new study created in memory with name: RotatE_HPO_Experiment
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.rotate.RotatE'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.stoppers.early_stopping:Inferred c

Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/6.00 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/22.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/22.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/22.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/22.0 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.75s seconds
INFO:pykeen.hpo.hpo:Pruned trial: <optuna.trial._trial.Trial object at 0x00000191792B96D0> at epoch 15 due to both.realistic.inverse_harmonic_mean_rank=0.0714797563597277
[I 2025-05-28 01:04:32,696] Trial 29 pruned. 


In [6]:
resultRotatE.save_to_directory('rotate_hpo_result')

# ComplEx Model

In [7]:
model_name = "ComplEx"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64),
        )
# Run the HPO pipeline
resultComplEx = run_pipeline(model_name, model_kwargs_range, 'ComplEx_HPO_Experiment')

[I 2025-05-28 01:04:32,829] A new study created in memory with name: ComplEx_HPO_Experiment
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.complex.ComplEx'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.SoftplusLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer)

Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/11.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/11.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/11.0 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/698 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/698 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 2.24s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early at epoch 100. The best result 0.07721246864922966 occurred at epoch 50.
INFO:pykeen.stoppers.early_stopping:Re-loading weights from best epoch from C:\Users\usuario\.data\pykeen\checkpoints\best-model-weights-d529187a-9e7d-47ee-be4a-d936d70ea336.pt


Evaluating on cuda:0:   0%|          | 0.00/2.79k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 2.15s seconds
[I 2025-05-28 02:29:28,679] Trial 29 finished with value: 0.07140259444713593 and parameters: {'model.embedding_dim': 192, 'optimizer.lr': 0.0004636928889952906, 'optimizer.weight_decay': 0.0002059667371964418, 'negative_sampler.num_negs_per_pos': 1, 'training.num_epochs': 150, 'training.batch_size': 16}. Best is trial 29 with value: 0.07140259444713593.


In [8]:
resultComplEx.save_to_directory('complex_hpo_result')

# ConvKB Model

In [ ]:
model_name = "ConvKB"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64),
            hidden_dropout_rate=dict(type="float", low=0.3, high=0.5, step=0.1),
            num_filters=dict(type="int", low=64, high=128, step=32),
        )
# Run the HPO pipeline
resultConvKB = run_pipeline(model_name, model_kwargs_range, 'ConvKB_HPO_Experiment')

[I 2025-05-28 11:53:18,425] A new study created in memory with name: ConvKB_HPO_Experiment
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.conv_kb.ConvKB'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using regularizer: <class 'pykeen.regularizers.LpRegularizer'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:

Training epochs on cuda:0:   0%|          | 0/150 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/44.0 [00:00<?, ?batch/s]

In [ ]:
resultConvKB.save_to_directory('convkb_hpo_result')